<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/greener_delhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arcgis

In [ ]:
%matplotlib inline

import pandas as pd
from datetime import datetime
from IPython.display import Image
import matplotlib.pyplot as plt

import arcgis
from arcgis.gis import GIS
from arcgis.raster.functions import apply, clip, remap, colormap
from arcgis.geocoding import geocode

In [ ]:
gis = GIS("https://www.arcgis.com", "girija2204", "Girija2204@")

In [ ]:
gis = GIS()

In [ ]:
landsat_data = gis.content.get("d9b466d6a9e647ce8d1dd5fe12eb434b")
landsat_data.layers

[<ImageryLayer url:"https://landsat2.arcgis.com/arcgis/rest/services/Landsat/MS/ImageServer">]

In [ ]:
landsat_data

<Item title:"Multispectral Landsat" type:Imagery Layer owner:esri>

In [ ]:
landsat_data.type

'Image Service'

In [ ]:
landsat_data.layers[0]

In [ ]:
for property in landsat_data.layers[0].properties:
  print(property)

In [ ]:
llayer = landsat_data.layers[0]
# print(llayer.properties['rasterFunctionInfos'][0].get('name'))
for rasterFunction in llayer.properties['rasterFunctionInfos']:
  print(rasterFunction.get('name'))

In [ ]:
apply(llayer,'Natural Color with DRA')

In [ ]:
apply(llayer, 'NDVI Colorized')

In [ ]:
boundaries = gis.content.get("3013e4533dc64a848159cafe7bc2de84")

In [ ]:
boundaries.url

In [ ]:
delhi_geocode = geocode("New Delhi, India", out_sr=llayer.properties.spatialReference)
len(delhi_geocode)
area = delhi_geocode[0]
area['extent']
# llayer.extent = area['extent']
# llayer

In [ ]:
state_boundaries = boundaries.layers[1]
delhi = state_boundaries.query(where="NAME='NCT of Delhi'")
delhi_geom = delhi.features[0].geometry
delhi_geom['spatialReference'] = {'wkid':4326}
# delhi_geom = delhi.features[0].geometry
# delhi_geom['spatialReference'] = {'wkid':4326}
# delhi.features[0].extent = area['extent']

In [ ]:
selected = llayer.filter_by(where="(Category=1) AND (CloudCover <= 0.05)",
                 time=[datetime(2015, 1, 1),datetime(2017, 12, 31)],
                 geometry=arcgis.geometry.filters.intersects(area['extent']))
df = selected.query(out_fields="AcquisitionDate, GroupName, CloudCover, DayOfYear", 
                    order_by_fields="AcquisitionDate").sdf
df['AcquisitionDate'] = pd.to_datetime(df['AcquisitionDate'], unit='ms')
df

In [ ]:
delhi_image = llayer.filter_by("OBJECTID=2752790") # 2017-10-15

In [ ]:
def apply_transformations(dated_layer):
  dated_layer_ndviraw = apply(dated_layer,"NDVI Raw")
  dated_layer_clipped = clip(dated_layer_ndviraw, delhi_geom)
  dated_layer_clipped.extent = area['extent']
  dated_layer_clipped_remapped = remap(dated_layer_clipped,
      input_ranges=[0.4,0.5, 0.5, 1],
      output_values=[1, 2])
  dated_layer_clipped_colormapped = colormap(dated_layer_clipped_remapped,colormap=[[1, 124, 252, 0], [2, 0, 102, 0]], astype='u8')
  return dated_layer_clipped_colormapped

In [ ]:
delhi_image_2927742 = llayer.filter_by("OBJECTID=2927742") # 2015-03-23
delhi_image_2678985 = llayer.filter_by("OBJECTID=2678985") # 2015-08-30
delhi_image_2752790 = llayer.filter_by("OBJECTID=2752790") # 2016-03-09
delhi_image_2945298 = llayer.filter_by("OBJECTID=2945298") # 2016-10-19
delhi_image_2974179 = llayer.filter_by("OBJECTID=2974179") # 2017-03-05
delhi_image_2095534 = llayer.filter_by("OBJECTID=2095534") # 2017-10-31

In [ ]:
cm_delhi_image_2927742 = apply_transformations(delhi_image_2927742)
cm_delhi_image_2678985 = apply_transformations(delhi_image_2678985)
cm_delhi_image_2752790 = apply_transformations(delhi_image_2752790)
cm_delhi_image_2945298 = apply_transformations(delhi_image_2945298)
cm_delhi_image_2974179 = apply_transformations(delhi_image_2974179)
cm_delhi_image_2095534 = apply_transformations(delhi_image_2095534)

In [ ]:
delhi_ndviraw = apply(delhi_image,"NDVI Raw")
delhi_clip = clip(delhi_ndviraw, delhi_geom)
delhi_clip.extent = area['extent']
# delhi_clip

In [ ]:
delhi_clip_remapped = remap(delhi_clip,
      input_ranges=[0.4,0.5, 0.5, 1],
      output_values=[1, 2])
delhi_clip_colormapped = colormap(delhi_clip_remapped,colormap=[[1, 124, 252, 0], [2, 0, 102, 0]], astype='u8')
delhi_clip_colormapped

In [ ]:
# delhi_clip_colormapped.draw_graph()

In [ ]:
map2 = gis.map('New Delhi, India')
map2.add_layer(delhi_clip_colormapped)
map2.legend = True
map2.export_to_html('gifg.html')

In [ ]:
def layer_export_to_html(layer,name):
  map = gis.map('New Delhi, India')
  map.add_layer(layer)
  map.legend = True
  map.export_to_html(f'{name}.html')

In [ ]:
layer_export_to_html(cm_delhi_image_2927742,'cm_delhi_image_2927742')
layer_export_to_html(cm_delhi_image_2678985,'cm_delhi_image_2678985')
layer_export_to_html(cm_delhi_image_2752790,'cm_delhi_image_2752790')
layer_export_to_html(cm_delhi_image_2945298,'cm_delhi_image_2945298')
layer_export_to_html(cm_delhi_image_2974179,'cm_delhi_image_2974179')
layer_export_to_html(cm_delhi_image_2095534,'cm_delhi_image_2095534')

In [ ]:
landsat_data.layers[0].properties

In [ ]:
gis.content.search("title: ScientificData/SeaTemperature", "Imagery Layer", outside_org=True)